<a href="https://colab.research.google.com/github/jarodchristiansen/Machine-Learning-Deep-Learning/blob/master/Speech_To_Speech_Translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unified Version 3

In [35]:
!apt-get -qq install ffmpeg
!pip install -q pydub SpeechRecognition
!pip install -q googletrans==4.0.0-rc1 gTTS


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
openai 1.75.0 requires httpx<1,>=0.23.0, but you have httpx 0.13.3 which is incompatible.
google-genai 1.10.0 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.13.3 which is incompatible.
langsmith 0.3.31 requires httpx<1,>=0.23.0, but

In [68]:
from IPython.display import display
import ipywidgets as widgets

# Map readable names to language codes
LANGUAGES = {
    "English": "en",
    "Spanish": "es",
    "French": "fr",
    "German": "de",
    "Hindi": "hi",
    "Mandarin (Simplified)": "zh-cn",
    "Russian": "ru",
}

source_dropdown = widgets.Dropdown(
    options=LANGUAGES.keys(),
    value="English",
    description="🎙 Source:"
)
target_dropdown = widgets.Dropdown(
    options=LANGUAGES.keys(),
    value="Spanish",
    description="🗣 Target:"
)

display(source_dropdown, target_dropdown)


Dropdown(description='🎙 Source:', options=('English', 'Spanish', 'French', 'German', 'Hindi', 'Mandarin (Simpl…

Dropdown(description='🗣 Target:', index=1, options=('English', 'Spanish', 'French', 'German', 'Hindi', 'Mandar…

In [31]:
from IPython.display import Javascript, display
from google.colab import output
from base64 import b64decode
from pydub import AudioSegment
import os

# Inject JS recorder function
RECORD_JS = """
async function record(duration=5000) {
  const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
  const recorder = new MediaRecorder(stream);
  const chunks = [];

  recorder.ondataavailable = e => chunks.push(e.data);
  recorder.start();
  await new Promise(r => setTimeout(r, duration));
  recorder.stop();

  await new Promise(r => recorder.onstop = r);
  const blob = new Blob(chunks, { type: 'audio/webm' });
  const reader = new FileReader();

  return await new Promise(resolve => {
    reader.onloadend = () => resolve(reader.result);
    reader.readAsDataURL(blob);
  });
}
"""

def record_audio(duration=5):
    print(f"🎙 Recording for {duration} seconds...")
    display(Javascript(RECORD_JS))
    data_url = output.eval_js(f"record({duration * 1000})")
    header, encoded = data_url.split(',', 1)

    with open("temp.webm", "wb") as f:
        f.write(b64decode(encoded))
    print("✅ Saved temp.webm")

    # Convert to WAV using pydub
    audio = AudioSegment.from_file("temp.webm", format="webm")
    audio = audio.set_frame_rate(16000).set_channels(1)
    audio.export("audio.wav", format="wav")
    print("🎉 Exported clean audio.wav")
    return "audio.wav"


In [34]:
import speech_recognition as sr

def transcribe_wav(wav_path):
    recognizer = sr.Recognizer()
    with sr.AudioFile(wav_path) as src:
        audio = recognizer.record(src)
        print("🔎 Transcribing...")
        try:
            text = recognizer.recognize_google(audio)
            print("📝 Transcript:", text)
            return text
        except sr.UnknownValueError:
            print("🤷 Google Speech Recognition could not understand the audio.")
        except sr.RequestError as e:
            print(f"🔌 Could not request results from Google: {e}")


🎙 Recording for 10 seconds...


<IPython.core.display.Javascript object>

✅ Saved temp.webm
🎉 Exported clean audio.wav
🔎 Transcribing...
📝 Transcript: hello how are you where is the airport


In [65]:
import re

def detect_intent(text):
    question_words = ['who', 'what', 'when', 'where', 'why', 'how', 'do', 'does', 'is', 'are', 'can', 'should']
    command_patterns = r"^(please\s)?(go|stop|send|open|close|turn|say|tell|start|shut|bring|make|show)\b"

    if text.strip().endswith('?') or text.split()[0].lower() in question_words:
        return "question"
    elif re.match(command_patterns, text.strip().lower()):
        return "command"
    else:
        return "statement"


In [66]:
def enhance_prosody(text, intent):
    if intent == "question":
        # Add a longer pause at the end and maybe slight emphasis
        return text.strip() + "?"
    elif intent == "command":
        # Add rhythm, emphasize key directive tone
        return re.sub(r'(\s)', '... ', text.strip()) + "!"
    else:  # Statement
        # Add regular sentence pacing
        return text.strip().replace(",", ",...").replace(".", "....")


In [67]:
from googletrans import Translator

# def translate_text(text):
#     src_lang = LANGUAGES[source_dropdown.value]
#     tgt_lang = LANGUAGES[target_dropdown.value]

#     translator = Translator()
#     print(f"🌍 Translating from {src_lang} → {tgt_lang}...")
#     result = translator.translate(text, src=src_lang, dest=tgt_lang)

#     # Add simple pause/emphasis heuristics
#     translated = result.text.replace(',', ',...').replace('.', '....')
#     print(f"📝 Translation: {translated}")
#     return translated, tgt_lang

def translate_text(text):
    src_lang = LANGUAGES[source_dropdown.value]
    tgt_lang = LANGUAGES[target_dropdown.value]

    intent = detect_intent(text)
    print(f"🧠 Detected intent: {intent}")

    translator = Translator()
    print(f"🌍 Translating {src_lang} → {tgt_lang}...")
    result = translator.translate(text, src=src_lang, dest=tgt_lang)
    translated = result.text

    # Enhance based on intent
    prosodic_text = enhance_prosody(translated, intent)
    print(f"🎭 Prosodic Translation: {prosodic_text}")
    return prosodic_text, tgt_lang



def translate_text_with_effects(text, target_lang='es'):
    # Simple rules to simulate natural rhythm/emphasis
    if text.endswith('?'):
        text = "¿" + text.capitalize()
    if ',' not in text and ' and ' in text:
        text = text.replace(' and ', ', and ')
    if '.' not in text:
        text += '.'

    translator = Translator()
    print(f"🌍 Translating with rhythm hints to {target_lang}...")
    result = translator.translate(text, dest=target_lang)
    translated = result.text

    # Add gTTS-friendly "pauses" if needed
    translated = translated.replace(',', ',...').replace('.', '....')
    print(f"📝 Translated with pauses: {translated}")
    return translated


In [59]:
from gtts import gTTS
from IPython.display import Audio

def speak_text(text, lang_code='es', filename='translated_audio.mp3'):
    print("🔊 Synthesizing speech...")
    tts = gTTS(text=text, lang=lang_code)
    tts.save(filename)
    print(f"✅ Saved {filename}")
    return Audio(filename)



In [70]:
wav = record_audio(duration=25)
text = transcribe_wav(wav)

if text:
    translated_text, tgt_lang = translate_text(text)
    display(Audio("audio.wav"))  # Original speech
    print(f"🔈 Transcript: {text}")
    print(f"🌍 Translation: {translated_text}")

    display(speak_text(translated_text, lang_code=tgt_lang))

🎙 Recording for 25 seconds...


<IPython.core.display.Javascript object>

✅ Saved temp.webm
🎉 Exported clean audio.wav
🔎 Transcribing...
📝 Transcript: hi could you please tell me how to get to the airport how are you where do we need to go why how start shut bring
🧠 Detected intent: statement
🌍 Translating en → es...
🎭 Prosodic Translation: Hola,............ ¿podrías decirme cómo llegar al aeropuerto? ¿Cómo estás? ¿A dónde debemos ir? ¿Por qué empezar?


🔈 Transcript: hi could you please tell me how to get to the airport how are you where do we need to go why how start shut bring
🌍 Translation: Hola,............ ¿podrías decirme cómo llegar al aeropuerto? ¿Cómo estás? ¿A dónde debemos ir? ¿Por qué empezar?
🔊 Synthesizing speech...
✅ Saved translated_audio.mp3


In [52]:
text

'where is the Apple'